In [47]:
# Imports
import pandas as pd
import numpy as np

In [48]:
# Load data
df_gbif = pd.read_csv("Allspecies_GBIFrecords_fieldnotes_filtered_final.csv")
df_hcnq = pd.read_csv("../../../HCNQ/HCNQ_dataset_filtered_final.csv")

In [49]:
df_gbif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46757 entries, 0 to 46756
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gbifID                    46757 non-null  int64  
 1   hcnqID                    0 non-null      float64
 2   institutionCode           46570 non-null  object 
 3   recordedBy                45255 non-null  object 
 4   eventDate                 46757 non-null  object 
 5   year                      46757 non-null  float64
 6   month                     46757 non-null  float64
 7   day                       46250 non-null  float64
 8   country                   46757 non-null  object 
 9   decimalLongitude          46757 non-null  float64
 10  decimalLatitude           46757 non-null  float64
 11  elevation                 34599 non-null  float64
 12  family                    46757 non-null  object 
 13  genus                     46757 non-null  object 
 14  specie

In [50]:
# Merge two datasets 
df_final = pd.concat([df_hcnq, df_gbif])

# Reset indices
df_final = df_final.reset_index(drop=True)

In [51]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53848 entries, 0 to 53847
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gbifID                    46757 non-null  float64
 1   hcnqID                    7091 non-null   float64
 2   institutionCode           53661 non-null  object 
 3   recordedBy                52304 non-null  object 
 4   eventDate                 53847 non-null  object 
 5   year                      53847 non-null  float64
 6   month                     53848 non-null  float64
 7   day                       53341 non-null  float64
 8   country                   53848 non-null  object 
 9   decimalLongitude          53848 non-null  float64
 10  decimalLatitude           53848 non-null  float64
 11  elevation                 41516 non-null  object 
 12  family                    53848 non-null  object 
 13  genus                     53800 non-null  object 
 14  specie

In [52]:
# Delete duplicates
dup_cols = ['acceptedScientificName', 'decimalLongitude', 'decimalLatitude','year', 'recordedBy']
df_final = df_final.drop(df_final.loc[:, dup_cols].dropna().duplicated().loc[lambda x: x].index)
df_final = df_final.drop_duplicates()

In [53]:
# Check if true duplicates were deleted 
(df_final.loc[:, dup_cols].dropna().duplicated().loc[lambda x: x].index).shape

(0,)

In [54]:
df_final[df_final.duplicated()]

,gbifID,hcnqID,institutionCode,recordedBy,eventDate,year,month,day,country,decimalLongitude,...,genus,species,acceptedScientificName,scientificNameAuthorship,image_url,reproductiveCondition,occurrenceRemarks,dynamicProperties,fieldNotes,FieldNotes


In [55]:
df_final.shape

(53799, 23)

In [56]:
# Reset indices of the dataframe
df_final = df_final.reset_index(drop=True)

In [57]:
# Add a column with a filter per year: before 1970, between 1970 and 2010, and since 2011
# Define a function for classifying records by year
def add_year_col(row):
    if row['year'] < 1970:
        val = '<1970'
    elif (row['year'] >= 1970) and (row['year'] <= 2010):
        val = '1970-2010'
    else:
        val = '>2011'
    return val

#create new column using the function above
df_final['Year_interval'] = df_final.apply(add_year_col, axis=1)

In [58]:
# Count the number of values of the new column
df_final['Year_interval'].value_counts()

1970-2010    41961
>2011         9489
<1970         2349
Name: Year_interval, dtype: int64

In [59]:
# Add a column with the number of records by species
df_final["n_records"] = pd.Series()

for i, row in df_final.iterrows():   
    count_temp = len(df_final[df_final["acceptedScientificName"] == row["acceptedScientificName"]])
    df_final.at[i,'n_records']=count_temp

C:\Users\sayalaruano\AppData\Local\Temp\ipykernel_25328\476362668.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_final["n_records"] = pd.Series()


In [60]:
# Verify the correct total number of records 
test = df_final.groupby(['acceptedScientificName', 'n_records']).size().reset_index(name='count')
test["count"].sum()

53799

In [61]:
# Add a column with the interval of the number of records: 0-10, 11-100, 101-500, >500
# Define a function for classifying records by ther numbers
def add_nrecords_col(row):
    if (row['n_records'] >= 0) and (row['n_records'] <=10):
        val = '0-10'
    elif (row['n_records'] >= 11) and (row['n_records'] <=100):
        val = '11-100'
    elif (row['n_records'] >= 101) and (row['n_records'] <=500):
        val = '101-500'
    else:
        val = '>500'
    return val

#create new column using the function above
df_final['nrecords_interval'] = df_final.apply(add_nrecords_col, axis=1)

In [62]:
# Verify the number of intervas by species
test1 = df_final.groupby(['acceptedScientificName', 'nrecords_interval']).size().reset_index(name='count')

In [63]:
# Count the number of values of the new column
test1["nrecords_interval"].value_counts()

11-100     241
0-10       106
101-500     49
>500        33
Name: nrecords_interval, dtype: int64

## Obtain binned latitude and longitue columns 

In [64]:
# Obtain the max latitude
max(df_final["decimalLatitude"])

39.732207

In [65]:
# Obtain the min latitude
min(df_final["decimalLatitude"])

-42.666667

In [66]:
# Create a list with the bins for the binned latitude column
start_lat = -50
n_lat = 50
interval_lat = 20
bins_lat = np.arange(start_lat, n_lat, interval_lat)

In [67]:
bins_lat

array([-50, -30, -10,  10,  30])

In [68]:
# Create a column with the binned latitude values
df_final['Binned_latitude'] = pd.cut(df_final['decimalLatitude'], bins_lat)

In [69]:
# Convert column type into object
df_final["Binned_latitude"] = df_final["Binned_latitude"].astype('string')

In [70]:
# Check binned latitude counts
df_final['Binned_latitude'].value_counts()

(-10.0, 10.0]     22172
(-30.0, -10.0]    19208
(10.0, 30.0]      12304
(-50.0, -30.0]       71
Name: Binned_latitude, dtype: Int64

In [71]:
# Obtain the max longitude
max(df_final["decimalLongitude"])

-26.26

In [72]:
# Obtain the min longitude
min(df_final["decimalLongitude"])

-159.667

In [73]:
# Create a list with the bins for the binned longitude column
start_long = -170
n_long = -30
interval_long = 20
bins_long = np.arange(start_long, n_long, interval_long)

In [74]:
bins_long

array([-170, -150, -130, -110,  -90,  -70,  -50])

In [75]:
# Create a column with the binned longitude values
df_final['Binned_longitude'] = pd.cut(df_final['decimalLongitude'], bins_long)

In [76]:
# Convert column type into object
df_final["Binned_longitude"] = df_final["Binned_longitude"].astype("string")

In [77]:
# Check binned longitude counts
df_final['Binned_longitude'].value_counts()

(-90.0, -70.0]      20813
(-110.0, -90.0]     10238
(-70.0, -50.0]       6800
(-130.0, -110.0]       37
(-170.0, -150.0]       36
Name: Binned_longitude, dtype: Int64

In [78]:
# Change the order of columns
df_final = df_final[['gbifID', 'hcnqID','institutionCode','recordedBy','eventDate', 'year',
        'month','day','Year_interval', 'country','decimalLongitude', 'Binned_longitude', 'decimalLatitude', 'Binned_latitude',
        'elevation','family','genus','species', 'acceptedScientificName',
        'scientificNameAuthorship','image_url','reproductiveCondition', 'n_records', 'nrecords_interval',
         'occurrenceRemarks', 'dynamicProperties', 'fieldNotes', 'FieldNotes']]


In [80]:
# Delete records that have NA in FieldNotes
# Getting indices
indices_del = df_final[(df_final.FieldNotes.isnull())].index
 
#droping row based on column value
df_final.drop(indices_del,inplace=True)

# Reset indices
df_final = df_final.reset_index(drop=True)

In [81]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53793 entries, 0 to 53792
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gbifID                    46739 non-null  float64
 1   hcnqID                    7054 non-null   float64
 2   institutionCode           53606 non-null  object 
 3   recordedBy                52250 non-null  object 
 4   eventDate                 53792 non-null  object 
 5   year                      53792 non-null  float64
 6   month                     53793 non-null  float64
 7   day                       53286 non-null  float64
 8   Year_interval             53793 non-null  object 
 9   country                   53793 non-null  object 
 10  decimalLongitude          53793 non-null  float64
 11  Binned_longitude          37918 non-null  string 
 12  decimalLatitude           53793 non-null  float64
 13  Binned_latitude           53749 non-null  string 
 14  elevat

In [82]:
# Export final dataset as csv
df_final.to_csv("Allspecies_GBIF_HCNQ_fieldnotes_final.csv", index=False)